In [1]:
import os
import cohere

co = cohere.Client(os.environ["COHERE_API_KEY"])

In [2]:
restaurant_dataset = co.datasets.create(
    name="restaurant_data",
    data=open("./restaurant_chat.jsonl", "rb"),
    type="chat-finetune-input",
)

print(restaurant_dataset.id)

restaurant-data-bav05h


In [4]:
ds = co.wait(restaurant_dataset)
print(ds)

dataset=Dataset(id='restaurant-data-bav05h', name='restaurant_data', created_at=datetime.datetime(2024, 6, 3, 11, 15, 45, 686668, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 6, 3, 11, 15, 45, 686668, tzinfo=datetime.timezone.utc), dataset_type='chat-finetune-input', validation_status='validated', validation_error=None, schema_='{"name":"cohere.chat_finetune_input","type":"record","fields":[{"name":"messages","type":{"type":"array","items":{"name":"cohere.message","type":"record","fields":[{"name":"role","type":"string"},{"name":"content","type":"string"}]}}},{"name":"is_eval","type":"boolean","default":false}]}', required_fields=None, preserve_fields=None, dataset_parts=[DatasetPart(id='593ebfde-9c64-4f46-a163-694eca02b27c', name='restaurant_chat.avro', url='https://storage.googleapis.com/cohere-production-user-datasets/dataset-api-temp/30b5ef85-e023-4a52-8366-4e8ce5598652/5e56f1fd-3535-4eb7-a3cc-86143148f374/restaurant-data-bav05h/000_restaurant_chat.avro?X-Goog-

In [5]:
from cohere.finetuning import FinetunedModel, Settings, BaseModel

In [7]:
finetuned_model = co.finetuning.create_finetuned_model(
    request=FinetunedModel(
        name="restaurant-chat-model",
        settings=Settings(
            base_model=BaseModel(
                base_type="BASE_TYPE_CHAT",
            ),
            dataset_id=restaurant_dataset.id,
        ),
    ),
)

In [8]:
ft = co.finetuning.get_finetuned_model(finetuned_model.finetuned_model.id)

In [10]:
prompt = f"""
You are tutor assistant that can give answers to the questions.

** Instructions ** 
Only provide information that lies in scope of restaurant.

Ensure that the questions which are outside the scope of restaurant gives message as *I am not trained to answer this question How ever you can ask me any question related to restaurant*.
"""

In [12]:
response = co.chat(
    message=prompt,
    model=ft.finetuned_model.id + "-ft",
    # optional (to specify a preamble)
    preamble="You are a chatbot trained to answer on restaurant booking related questions only.",
    # optional
    return_prompt=True,
)

In [13]:
new_response = co.chat(
    message="Who is the PM of India?",
    model=ft.finetuned_model.id + "-ft",
)

In [14]:
print(new_response.text)

The current Prime Minister of India is Narendra Modi. He has held the office since May 2014. Narendra Modi belongs to the Bharatiya Janata Party (BJP). He is the 14th Prime Minister of India.
